In this notebook we'll load the bridnet model, load the raw files, generate embeddings and make predictions on the embeddings

In [5]:
import numpy as np
import pandas as pd
import os

In [6]:
embedding_folder = '/home/jupyter/research/models'

In [7]:
test_embedding_folder = '/home/jupyter/research/test_embeddings'

In [8]:
raw_df = pd.read_csv('/home/jupyter/research/raw_data.csv')
raw_df = raw_df.rename(columns={"filename": "Filename"})

In [11]:
model_names = ['bvc', 'birdnet-realtime']


In [12]:
def load_embeddings_labels(model_folder,model_name):
    embeddings = np.load(os.path.join(model_folder,model_name+'_embeddings.npy'))
    labels = np.load(os.path.join(model_folder,model_name+'_labels.npy'))
    return embeddings, labels

In [13]:
model_embeddings = {}
model_labels = {}

for model in model_names:
    model_embeddings[model],model_labels[model] = load_embeddings_labels(embedding_folder,model)
    

In [14]:
unk_filenames = np.load(os.path.join(test_embedding_folder,'unk_filenames.npy'))

In [15]:
unk_embeddings = {}

for model in model_names:
    unk_embeddings[model],_ = load_embeddings_labels(test_embedding_folder,model)

In [116]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.preprocessing import normalize,StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import rbf_kernel



# Function to normalize embeddings using L2 normalization
def normalize_embeddings(embeddings):
    return normalize(embeddings, axis=1, norm='l2')

# Function to convert similarity scores to probabilities using softmax
def softmax(scores):
    exp_scores = np.exp(scores - np.max(scores, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

# Calculate prototypes using median or trimmed mean
def calculate_prototypes(embeddings, labels, use_median=True):
    unique_labels = np.unique(labels)
    prototypes = {}
    for label in unique_labels:
        label_embeddings = embeddings[labels == label]
        if use_median:
            prototypes[label] = np.median(label_embeddings, axis=0, keepdims=True)
        else:
            # Trimmed mean: remove the highest and lowest 10% of values
            sorted_embeddings = np.sort(label_embeddings, axis=0)
            trim_fraction = int(0.1 * label_embeddings.shape[0])
            trimmed_embeddings = sorted_embeddings[trim_fraction:-trim_fraction]
            prototypes[label] = np.mean(trimmed_embeddings, axis=0, keepdims=True)
    return prototypes


def calculate_prototype_matrix(prototypes):
    # Flatten prototype dictionary to arrays
    prototype_labels = []
    prototype_matrix = []
    for label, center in prototypes.items():
        prototype_labels.append(label)
        prototype_matrix.append(center)
    prototype_matrix = np.array(prototype_matrix).squeeze()
    return prototype_matrix,prototype_labels



def bird_classifier(target_embeddings, prototypes,unknown_files, threshold=0.24):
    
    prototype_matrix,prototype_labels = calculate_prototype_matrix(prototypes)
    
    # Compute cosine similarity between new embeddings and prototypes
    similarities = cosine_similarity(target_embeddings, prototype_matrix)

    # Convert similarities to probabilities
    probabilities = softmax(similarities)

    # Assign labels and calculate confidence scores based on the highest probability
    predicted_labels = []
    confidence_scores = []
    for prob in probabilities:
        max_prob = np.max(prob)
        if max_prob >= threshold:
            predicted_label = prototype_labels[np.argmax(prob)]
        else:
            predicted_label = 'unknown'
        predicted_labels.append(predicted_label)
        confidence_scores.append(max_prob)

    # Create a DataFrame to store the results
    results_df = pd.DataFrame({
        'Filename': [filename.split('/')[-1] for filename in unknown_files],
        'Predicted Label': predicted_labels,
        'Confidence Score': confidence_scores,
        'Probabilities': list(probabilities)
    })
    return results_df

def make_prediction(known_embeddings,known_labels,unknown_embeddings,unknown_files,n_components=142):
    
    # normalized_known_embeddings = normalize_embeddings(known_embeddings)
    # scaler = StandardScaler()
    # known_embeddings = scaler.fit_transform(known_embeddings)

    # Apply Truncated SVD to reduce dimensionality
    # reduce = TruncatedSVD(n_components=n_components)  # Reduce to 50 dimensions
    reduce = PCA(n_components=n_components)  # Reduce to 50 dimensions

    reduced_embeddings = reduce.fit_transform(known_embeddings)
    
    # Normalize known embeddings
    reduced_embeddings = normalize_embeddings(reduced_embeddings)

    prototypes = calculate_prototypes(reduced_embeddings, known_labels)

    # unknown_embeddings = normalize_embeddings(unknown_embeddings)

    # Example new bird sound embeddings (to classify)
    # unknown_embeddings = scaler.transform(unknown_embeddings)

    reduced_new_embeddings = reduce.transform(unknown_embeddings)
    reduced_new_embeddings = normalize_embeddings(reduced_new_embeddings)
    
    # results_df = similarity_based_classifier(reduced_new_embeddings,prototypes,unknown_files)


    results_df = bird_classifier(reduced_new_embeddings,prototypes,unknown_files)
    return results_df



In [117]:
df = {}
for model in model_names:
    df[model] = make_prediction(model_embeddings[model],model_labels['bvc'],unk_embeddings[model],unk_filenames)

In [118]:
df['bvc']

,Filename,Predicted Label,Confidence Score,Probabilities
0,Magiagi Feb 2022_20211115_160004_17930_18450.wav,crimson_crowned_fruit_dove,0.277438,"[0.27743775, 0.11521832, 0.14347143, 0.1224848..."
1,139904_Macaulay_Doug Pratt_96120_96920.wav,white_throated_pigeon,0.268487,"[0.15248972, 0.11578435, 0.21266748, 0.1130581..."
2,Magiagi Feb 2022_20211115_160004_138210_138720...,crimson_crowned_fruit_dove,0.264560,"[0.26455992, 0.10610789, 0.13001342, 0.1298450..."
3,161361541 Macaulay_Diana Novoa_42810_44100.wav,tooth_billed_pigeon,0.261013,"[0.18510735, 0.10373467, 0.15824708, 0.1006644..."
4,Magiagi Feb 2022_20211115_160004_109410_110410...,tooth_billed_pigeon,0.276027,"[0.17632392, 0.10116738, 0.16299866, 0.1031605..."
...,...,...,...,...
89,Magiagi Feb 2022_20211115_160004_28430_29510.wav,crimson_crowned_fruit_dove,0.281415,"[0.2814147, 0.104444325, 0.13445048, 0.1118944..."
90,Magiagi Feb 2022_20211115_160004_78730_80730.wav,unknown,0.227571,"[0.22757062, 0.107619666, 0.20281696, 0.108612..."
91,Magiagi Feb 2022_20211115_160004_117040_118210...,unknown,0.222624,"[0.21138287, 0.12065978, 0.16698669, 0.1271354..."
92,Magiagi Feb 2022_20211115_160004_129690_130380...,crimson_crowned_fruit_dove,0.254897,"[0.2548968, 0.116559856, 0.13331836, 0.1210505..."


In [119]:
df['bvc'][df['bvc']['Predicted Label'] == 'tooth_billed_pigeon']

,Filename,Predicted Label,Confidence Score,Probabilities
3,161361541 Macaulay_Diana Novoa_42810_44100.wav,tooth_billed_pigeon,0.261013,"[0.18510735, 0.10373467, 0.15824708, 0.1006644..."
4,Magiagi Feb 2022_20211115_160004_109410_110410...,tooth_billed_pigeon,0.276027,"[0.17632392, 0.10116738, 0.16299866, 0.1031605..."
11,161361541 Macaulay_Diana Novoa_13480_14890.wav,tooth_billed_pigeon,0.246194,"[0.17933926, 0.102311246, 0.17051718, 0.096012..."
17,Salelologa Aug 2021_f1842340352_780_1730.wav,tooth_billed_pigeon,0.257961,"[0.15003756, 0.09790067, 0.15818079, 0.0983597..."
27,Salelologa Aug 2021_f1842340352_3160_4500.wav,tooth_billed_pigeon,0.246591,"[0.15162341, 0.096397795, 0.1867544, 0.0914612..."
32,Salelologa Aug 2021_f1842340352_6060_7340.wav,tooth_billed_pigeon,0.298887,"[0.14790006, 0.102443695, 0.15574147, 0.097238..."
67,139904_Macaulay_Doug Pratt_69250_70370.wav,tooth_billed_pigeon,0.240034,"[0.17254594, 0.117148295, 0.15330055, 0.113505..."
74,139904_Macaulay_Doug Pratt_32360_33550.wav,tooth_billed_pigeon,0.249185,"[0.14940548, 0.09882024, 0.19414389, 0.0938701..."
75,139904_Macaulay_Doug Pratt_19130_20400.wav,tooth_billed_pigeon,0.288199,"[0.16448109, 0.107022285, 0.14889812, 0.105326..."
80,139904_Macaulay_Doug Pratt_48450_49650.wav,tooth_billed_pigeon,0.275901,"[0.16962472, 0.11029375, 0.14702696, 0.1083872..."


In [120]:
# df['birdnet-realtime'][:50]

In [121]:
# raw_df

In [122]:
def tbp_files(raw,pred):
    df = raw.merge(pred, how='inner', on='Filename')
    # return df
    return df[df['Predicted Label'] == 'tooth_billed_pigeon']

In [123]:
tbp_files(raw_df,df['bvc'])

,Filename,label,newname,Predicted Label,Confidence Score,Probabilities
0,Salelologa Aug 2021_f1842340352_6060_7340.wav,TBP,file2.wav,tooth_billed_pigeon,0.298887,"[0.14790006, 0.102443695, 0.15574147, 0.097238..."
1,Salelologa Aug 2021_f1842340352_780_1730.wav,TBP,file15.wav,tooth_billed_pigeon,0.257961,"[0.15003756, 0.09790067, 0.15818079, 0.0983597..."
2,Magiagi Feb 2022_20211115_160004_109410_110410...,TBP,file29.wav,tooth_billed_pigeon,0.276027,"[0.17632392, 0.10116738, 0.16299866, 0.1031605..."
3,139904_Macaulay_Doug Pratt_48450_49650.wav,TBP,file27.wav,tooth_billed_pigeon,0.275901,"[0.16962472, 0.11029375, 0.14702696, 0.1083872..."
4,139904_Macaulay_Doug Pratt_32360_33550.wav,TBP,file6.wav,tooth_billed_pigeon,0.249185,"[0.14940548, 0.09882024, 0.19414389, 0.0938701..."
10,Salelologa Aug 2021_f1842340352_3160_4500.wav,PIP,file16.wav,tooth_billed_pigeon,0.246591,"[0.15162341, 0.096397795, 0.1867544, 0.0914612..."


In [124]:
tbp_files(raw_df,df['birdnet-realtime'])

,Filename,label,newname,Predicted Label,Confidence Score,Probabilities
0,Salelologa Aug 2021_f1842340352_6060_7340.wav,TBP,file2.wav,tooth_billed_pigeon,0.270024,"[0.11682873, 0.09451293, 0.2550566, 0.08368677..."
1,Salelologa Aug 2021_f1842340352_780_1730.wav,TBP,file15.wav,tooth_billed_pigeon,0.250763,"[0.13465588, 0.09215642, 0.21721469, 0.0807294..."
2,Magiagi Feb 2022_20211115_160004_109410_110410...,TBP,file29.wav,tooth_billed_pigeon,0.244597,"[0.18722245, 0.10237674, 0.1648269, 0.09557710..."
3,139904_Macaulay_Doug Pratt_48450_49650.wav,TBP,file27.wav,tooth_billed_pigeon,0.245682,"[0.1756551, 0.10736873, 0.17343406, 0.10600608..."
26,Magiagi Feb 2022_20211115_160004_23500_25500.wav,CCFD,file8.wav,tooth_billed_pigeon,0.249449,"[0.21512276, 0.1030356, 0.18075225, 0.09419096..."


In [125]:
df['birdnet-realtime'][df['birdnet-realtime']['Predicted Label'] == 'tooth_billed_pigeon']

,Filename,Predicted Label,Confidence Score,Probabilities
4,Magiagi Feb 2022_20211115_160004_109410_110410...,tooth_billed_pigeon,0.244597,"[0.18722245, 0.10237674, 0.1648269, 0.09557710..."
17,Salelologa Aug 2021_f1842340352_780_1730.wav,tooth_billed_pigeon,0.250763,"[0.13465588, 0.09215642, 0.21721469, 0.0807294..."
32,Salelologa Aug 2021_f1842340352_6060_7340.wav,tooth_billed_pigeon,0.270024,"[0.11682873, 0.09451293, 0.2550566, 0.08368677..."
42,Magiagi Feb 2022_20211115_160004_23500_25500.wav,tooth_billed_pigeon,0.249449,"[0.21512276, 0.1030356, 0.18075225, 0.09419096..."
80,139904_Macaulay_Doug Pratt_48450_49650.wav,tooth_billed_pigeon,0.245682,"[0.1756551, 0.10736873, 0.17343406, 0.10600608..."


In [521]:
df['bvc'][df['bvc']['Filename'] == 'WEST UAFATO SM304491_0+1_20160323_093808trim_21210_22500.wav']['Probabilities'].tolist()

[array([0.1169361 , 0.08716074, 0.28852677, 0.07849108, 0.18980937,
        0.23907593], dtype=float32)]

In [104]:
np.unique(model_labels['bvc'])

array(['crimson_crowned_fruit_dove', 'honeyeater',
       'pacific_imperial_pigeon', 'samoan_starling',
       'tooth_billed_pigeon', 'white_throated_pigeon'], dtype='<U26')

In [29]:
true_label = np.array([0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0])

In [35]:
pred_label_perch = np.array([0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0])

In [40]:
pred_label_birdnet = np.array([0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0])

In [48]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score


# Function to calculate and print metrics
def calculate_metrics(true_labels, predicted_labels, method_name):
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    print(f"Metrics for {method_name}:")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print()

In [49]:
# Calculate metrics for pred_label_perch
calculate_metrics(true_label, pred_label_perch, "Perch")

# Calculate metrics for pred_label_birdnet
calculate_metrics(true_label, pred_label_birdnet, "BirdNET")

Metrics for Perch:
Precision: 0.50
Recall: 0.75
F1 Score: 0.60
Accuracy: 0.86

Metrics for BirdNET:
Precision: 0.40
Recall: 0.50
F1 Score: 0.44
Accuracy: 0.82

